In [222]:
from urllib import request
import nltk.data

import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
url = "http://www.gutenberg.org/cache/epub/1661/pg1661.txt"
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
import operator

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ajaymudhai/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ajaymudhai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Q.1 and Q.2 Downloading dataset and removing unnecessary symbols.

In [223]:
response = request.urlopen(url)
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters, PunktLanguageVars
raw = response.read().decode('utf8')
text=raw.replace('\n',' ').replace('\r',' ').replace('\t',' ').replace(',',' ').replace("''"," ")


# Q.2 Senetence Tokenizing and dividing sentences into training and test data.

In [224]:
from nltk.tokenize import sent_tokenize
sent_tokenize_list = sent_tokenize(text)
l=len(sent_tokenize_list)
print(l)

6960


In [225]:
"""Adding <s> and </s> to mark start and end of sentence"""
for i in range(l):
    sent_tokenize_list[i]="<s>"+sent_tokenize_list[i]+"</s>"
    

In [226]:
training_dataset=sent_tokenize_list[:5568]
test_dataset=sent_tokenize_list[5568:]



#  Unigram, bigram, trigrams and quadgrams.

In [227]:
from nltk.util import ngrams
from collections import Counter
token = nltk.word_tokenize(text)
token = nltk.word_tokenize(text)

unigrams = ngrams(text.split(), 1)

bigrams = ngrams(text.split(), 2)
trigrams = ngrams(text.split(), 3)
quadgrams = ngrams(text.split(), 4)

uf=dict()
vocab=0
for x in unigrams:
    if x not in uf:
        uf[x]=1
        vocab+=1
    else:
        uf[x]+=1
        vocab+=1

bf=dict()
bfl=0
for x in bigrams:
    if x not in bf:
        bf[x]=1
        bfl+=1
    else:
        bf[x]+=1
        bfl+=1

tf=dict()
for x in trigrams:
    if x not in tf:
        tf[x]=1
    else:
        tf[x]+=1

qf=dict()
for x in quadgrams:
    if x not in qf:
        qf[x]=1
    else:
        qf[x]+=1




# MLE for Unigram(probabilities of individual words)


In [237]:
freq_1gram = nltk.FreqDist(token)

len_corpus = len(token)
def unigram_prob(word):

    return freq_1gram[ word] / len_corpus
nl=[]
for x in token:
    nl.append(unigram_prob(x))
for i in range(5):
    print(token[i],nl[i])


﻿Project 8.238589553468446e-06
Gutenberg 0.00023891909705058494
's 0.0029988465974625142
The 0.0028505519855000824
Adventures 2.4715768660405338e-05


# MLE for bigram 

In [241]:
bmle={}
for x in bf:
    bmle[x]=bf[x]/uf[(x[0],)]
bl=list(bmle)

for i in range(5):
    print(bl[i],bmle[bl[i]])

('\ufeffProject', "Gutenberg's") 1.0
("Gutenberg's", 'The') 1.0
('The', 'Adventures') 0.01098901098901099
('Adventures', 'of') 1.0
('of', 'Sherlock') 0.0018355359765051395


# MLE for trigram 


In [243]:
tmle={}
for x in tf:
    tmle[x]=tf[x]/bf[(x[0],x[1])]
tl=list(tmle)

for i in range(5):
    print(tl[i],tmle[tl[i]])

('\ufeffProject', "Gutenberg's", 'The') 1.0
("Gutenberg's", 'The', 'Adventures') 1.0
('The', 'Adventures', 'of') 1.0
('Adventures', 'of', 'Sherlock') 1.0
('of', 'Sherlock', 'Holmes') 0.6


# MLE for quadgram

In [244]:
qmle={}
for x in qf:
    qmle[x]=qf[x]/tf[(x[0],x[1],x[2])]
ql=list(qmle)

for i in range(5):
    print(ql[i],qmle[ql[i]])

('\ufeffProject', "Gutenberg's", 'The', 'Adventures') 1.0
("Gutenberg's", 'The', 'Adventures', 'of') 1.0
('The', 'Adventures', 'of', 'Sherlock') 1.0
('Adventures', 'of', 'Sherlock', 'Holmes') 1.0
('of', 'Sherlock', 'Holmes', 'by') 0.6666666666666666


# Q.5 Add-1 Smoothing

In [249]:
addone={}
for x in bf:
    addone[x]=(bf[x]+1)/(uf[(x[0],)]+vocab)
    
al=list(addone)
bfl=list(bf)
for i in range(5):
    print(bfl[i])
    print("Bigram MLE before smoothing : %f"%bf[bfl[i]])
    print("After addone smoothing : %f"%addone[al[i]])
    


('\ufeffProject', "Gutenberg's")
Bigram MLE before smoothing : 1.000000
After addone smoothing : 0.000018
("Gutenberg's", 'The')
Bigram MLE before smoothing : 1.000000
After addone smoothing : 0.000018
('The', 'Adventures')
Bigram MLE before smoothing : 3.000000
After addone smoothing : 0.000037
('Adventures', 'of')
Bigram MLE before smoothing : 3.000000
After addone smoothing : 0.000037
('of', 'Sherlock')
Bigram MLE before smoothing : 5.000000
After addone smoothing : 0.000054


### Significant change can be seen after Add-1 Smoothing.

# Q.6 Good-turing smoothing 

In [232]:
fd={}
for x in bf.values():
    if x not in fd:
        fd[x]=1
    else:
        fd[x]+=1
fd[0]=bfl

gts={}
for i in range(10,-1,-1):
    gts[i]=((i + 1) * fd[i + 1]) / fd[i]

for i in range(10):
    print(i,gts[i],i-gts[i])

0 0.4375450862882193 -0.4375450862882193
1 0.2527584020291693 0.7472415979708307
2 1.0727546412443552 0.9272453587556448
3 1.8858746492048644 1.1141253507951356
4 3.0307539682539684 0.9692460317460316
5 4.016366612111293 0.9836333878887071
6 4.58679706601467 1.41320293398533
7 6.17910447761194 0.8208955223880601
8 6.434782608695652 1.5652173913043477
9 9.324324324324325 -0.32432432432432456


In [233]:
D=0
for i in range(10):
    D=D+i-gts[i]
D=D/10
D

0.7778938164221545

### d= 0.77